In [635]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer

In [636]:
df = pd.read_csv('one_hot_encode.csv')

In [637]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\edipa\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\edipa\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\edipa\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [638]:
df.head(1)

,title,rated,released,runtime,director,writer,actors,plot,language,country,...,Jessica Chastain,Chris Evans,Mark Ruffalo,Nicolas Cage,Chloë Grace Moretz,Johnny Depp,Kristen Stewart,Amy Adams,Robert De Niro,Chris Pine
0,Inception,PG-13,2010-07-16,148,Christopher Nolan,Christopher Nolan,"Leonardo DiCaprio, Joseph Gordon-Levitt, Ellio...",A thief who steals corporate secrets through t...,"English, Japanese, French","United Kingdom, United States",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [639]:
df["rated"].fillna("NA", inplace = True)
df["director"].fillna("NA", inplace = True)
df["writer"].fillna("NA", inplace = True)
df["language"].fillna("NA", inplace = True)
df["country"].fillna("NA", inplace = True)
df["awards"].fillna("NA", inplace = True)
df["metascore"].fillna(0, inplace = True)
df["dvd"].fillna("NA", inplace = True)
df["boxoffice"].fillna(0, inplace = True)
df["budget"].fillna(0, inplace = True)
df["gross"].fillna(0, inplace = True)


In [640]:
df.fillna(0, inplace=True)

In [641]:
rat = df.rated.value_counts()
rat

R            812
PG-13        596
PG           196
Not Rated    114
NA            21
G             11
Unrated        7
TV-14          6
TV-MA          6
NC-17          3
TV-PG          2
Name: rated, dtype: int64

In [642]:
df.loc[df['rated']=='TV-MA', 'rated'] = 'R'
df.loc[df['rated']=='NC-17', 'rated'] = 'R'
df.loc[df['rated']=='TV-14', 'rated'] = 'PG-13'
df.loc[df['rated']=='TV-PG', 'rated'] = 'PG'
df.loc[df['rated']=='Unrated', 'rated'] = 'NA'
df.loc[df['rated']=='Not Rated', 'rated'] = 'NA'

In [643]:
rat = df.rated.value_counts()
rat

R        821
PG-13    602
PG       198
NA       142
G         11
Name: rated, dtype: int64

In [644]:
stars = df.rated.value_counts()

In [645]:
st = df.star.value_counts()

In [646]:
stars_filter = st[st< 10]
stars_filter

Nicolas Cage            9
Jason Bateman           9
Michael Fassbender      9
Tom Cruise              9
Denzel Washington       9
                       ..
Darío Grandinetti       1
Dan Stevens             1
Judith Hoag             1
Paul Walker             1
Anne-Élisabeth Bossé    1
Name: star, Length: 922, dtype: int64

In [647]:
stars_replace = st[st<10].index.to_list()
stars_replace


['Nicolas Cage',
 'Jason Bateman',
 'Michael Fassbender',
 'Tom Cruise',
 'Denzel Washington',
 'Jennifer Lawrence',
 'Chris Hemsworth',
 'Dwayne Johnson',
 'Joseph Gordon-Levitt',
 'Johnny Depp',
 'Ben Stiller',
 'Gerard Butler',
 'Keira Knightley',
 'Ryan Reynolds',
 'Ben Affleck',
 'Ewan McGregor',
 'Anna Kendrick',
 'Tom Hanks',
 'Brad Pitt',
 'Tom Hardy',
 'Bradley Cooper',
 'Christian Bale',
 'Keanu Reeves',
 'Vin Diesel',
 'Robert Pattinson',
 'Jake Gyllenhaal',
 'Hugh Jackman',
 'Jason Statham',
 'Helen Mirren',
 'Joaquin Phoenix',
 'Leonardo DiCaprio',
 'James Franco',
 'Russell Crowe',
 'Will Ferrell',
 'Melissa McCarthy',
 'Chris Pine',
 "Dylan O'Brien",
 'Shailene Woodley',
 'Ryan Gosling',
 'Will Smith',
 'Eddie Redmayne',
 'Ethan Hawke',
 'Zac Efron',
 'Kristen Stewart',
 'Rachel Weisz',
 'Sylvester Stallone',
 'Reese Witherspoon',
 'Jack Black',
 'Meryl Streep',
 'Kevin Costner',
 'Owen Wilson',
 'James McAvoy',
 'Jennifer Aniston',
 'Daniel Craig',
 'Naomi Watts',
 'Mia

In [648]:
for star in stars_replace:
  df['star'] = df['star'].replace(star, 'other')

df['star'].value_counts()

other                  1673
Mark Wahlberg            13
Steve Carell             12
Liam Neeson              12
Matt Damon               12
Matthew McConaughey      12
Robert Downey Jr.        10
Adam Sandler             10
Jesse Eisenberg          10
Channing Tatum           10
Name: star, dtype: int64

# Plot Vector Transformation

In [649]:
Df_vector = df

In [650]:
Df_vector = Df_vector.replace(to_replace=',',value='', regex = True)

In [651]:
Df_vector['plot'] = Df_vector['plot'].str.replace(r'[^\w\s]+', '')

C:\Users\edipa\AppData\Local\Temp/ipykernel_8420/3461421690.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  Df_vector['plot'] = Df_vector['plot'].str.replace(r'[^\w\s]+', '')


In [652]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
stop_words=stopwords.words('english')
from nltk.tokenize.treebank import TreebankWordDetokenizer


In [653]:
lemmatizer=WordNetLemmatizer()
i = 0
Df_vector['new']=''
for  column in Df_vector['plot']:
    word_tokens = word_tokenize(column)      
    filtered_sentence = [] 
    for w in word_tokens:
        if w not in stop_words:
            filtered_sentence.append(lemmatizer.lemmatize(w))
    s =TreebankWordDetokenizer().detokenize(filtered_sentence)
    Df_vector.at[i,'new']=s    
    i = i + 1

In [654]:
Df_vector.head(2)

,title,rated,released,runtime,director,writer,actors,plot,language,country,...,Chris Evans,Mark Ruffalo,Nicolas Cage,Chloë Grace Moretz,Johnny Depp,Kristen Stewart,Amy Adams,Robert De Niro,Chris Pine,new
0,Inception,PG-13,2010-07-16,148,Christopher Nolan,Christopher Nolan,Leonardo DiCaprio Joseph Gordon-Levitt Elliot ...,A thief who steals corporate secrets through t...,English Japanese French,United Kingdom United States,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,A thief steal corporate secret use dreamsharin...
1,Iron Man 2,PG-13,2010-05-07,124,Jon Favreau,Justin Theroux Stan Lee Don Heck,Robert Downey Jr. Mickey Rourke Gwyneth Paltrow,With the world now aware of his identity as Ir...,English French Russian,United States,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,With world aware identity Iron Man Tony Stark ...


In [655]:
Df_vector['Vectorized'] = 'default value'
vectorizer = CountVectorizer()
for i in range(0,len(Df_vector)):
    x = vectorizer.fit_transform([Df_vector['new'][i]])
    Df_vector['Vectorized'][i] = x.toarray()

C:\Users\edipa\AppData\Local\Temp/ipykernel_8420/2256096206.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Df_vector['Vectorized'][i] = x.toarray()


In [656]:
Df_vector['new'][0]

'A thief steal corporate secret use dreamsharing technology given inverse task planting idea mind CEO tragic past may doom project team disaster'

In [657]:
Df_vector['plot'][0]

'A thief who steals corporate secrets through the use of dreamsharing technology is given the inverse task of planting an idea into the mind of a CEO but his tragic past may doom the project and his team to disaster'

In [658]:
x = vectorizer.fit_transform(Df_vector['new'])
df3 = pd.DataFrame(x.toarray())
df3.columns = vectorizer.get_feature_names_out()
df3

,007,10,100,100th,11,11yearold,12,12foot,12hour,12yearold,...,zequi,zero,zeus,zibby,zombie,zone,zoo,zookeeper,zoologist,zuckerberg
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1769,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1770,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1771,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1772,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [659]:
df3['thief'].sum()

11

In [660]:
print("Vocabulary: ", vectorizer.vocabulary_)

Vocabulary:  {'thief': 6828, 'steal': 6459, 'corporate': 1567, 'secret': 6001, 'use': 7256, 'dreamsharing': 2126, 'technology': 6742, 'given': 2949, 'inverse': 3631, 'task': 6721, 'planting': 5144, 'idea': 3409, 'mind': 4430, 'ceo': 1191, 'tragic': 6958, 'past': 4993, 'may': 4307, 'doom': 2089, 'project': 5337, 'team': 6734, 'disaster': 1996, 'with': 7563, 'world': 7599, 'aware': 639, 'identity': 3416, 'iron': 3659, 'man': 4207, 'tony': 6910, 'stark': 6442, 'must': 4596, 'contend': 1534, 'declining': 1817, 'health': 3182, 'vengeful': 7301, 'mad': 4159, 'tie': 6870, 'father': 2601, 'legacy': 3976, 'in': 3469, '1954': 51, 'us': 7254, 'marshal': 4266, 'investigates': 3634, 'disappearance': 1990, 'murderer': 4584, 'escaped': 2390, 'hospital': 3344, 'criminally': 1645, 'insane': 3546, 'magically': 4174, 'realistic': 5509, 'version': 7312, 'toronto': 6922, 'young': 7657, 'defeat': 1827, 'new': 4670, 'girlfriend': 2944, 'seven': 6095, 'evil': 2433, 'ex': 2438, 'one': 4811, 'order': 4848, 'win

# Cleaning plot columns

In [661]:
list = df3.columns.to_list()

In [662]:
list

['007',
 '10',
 '100',
 '100th',
 '11',
 '11yearold',
 '12',
 '12foot',
 '12hour',
 '12yearold',
 '13',
 '14',
 '14yearold',
 '150',
 '15yearold',
 '16',
 '1600',
 '1630s',
 '16th',
 '16year',
 '16yearold',
 '17yearold',
 '18',
 '1800s',
 '1820',
 '1820s',
 '1825',
 '1845',
 '1850s',
 '1870s',
 '1890s',
 '18th',
 '18thcentury',
 '19',
 '1904',
 '1912',
 '1917',
 '1918',
 '1920s',
 '1927',
 '1930s',
 '1931',
 '1936',
 '1940',
 '1942',
 '1945',
 '1947',
 '1949',
 '1950s',
 '1952',
 '1953',
 '1954',
 '1956',
 '1957',
 '1959',
 '1960',
 '1960s',
 '1961',
 '1962',
 '1964',
 '1965',
 '1966',
 '1967',
 '1968',
 '1969',
 '1970',
 '1970s',
 '1971',
 '1972',
 '1973',
 '1974',
 '1976',
 '1977',
 '1978',
 '1979',
 '1980',
 '1980s',
 '1981',
 '1982',
 '1984',
 '1985',
 '1986',
 '1987',
 '1988',
 '1989',
 '1990s',
 '1990sera',
 '1993',
 '1996',
 '1997',
 '1998',
 '19th',
 '19thcentury',
 '19yearold',
 '20',
 '2001',
 '2003',
 '2004',
 '2005',
 '20062007',
 '2008',
 '2009',
 '2010',
 '2011',
 '2013',

In [663]:
df3.drop(['007',
 '10',
 '100',
 '100th',
 '11',
 '11yearold',
 '12',
 '12foot',
 '12hour',
 '12yearold',
 '13',
 '14',
 '14yearold',
 '150',
 '15yearold',
 '16',
 '1600',
 '1630s',
 '16th',
 '16year',
 '16yearold',
 '17yearold',
 '18',
 '1800s',
 '1820',
 '1820s',
 '1825',
 '1845',
 '1850s',
 '1870s',
 '1890s',
 '18th',
 '18thcentury',
 '19',
 '1904',
 '1912',
 '1917',
 '1918',
 '1920s',
 '1927',
 '1930s',
 '1931',
 '1936',
 '1940',
 '1942',
 '1945',
 '1947',
 '1949',
 '1950s',
 '1952',
 '1953',
 '1954',
 '1956',
 '1957',
 '1959',
 '1960',
 '1960s',
 '1961',
 '1962',
 '1964',
 '1965',
 '1966',
 '1967',
 '1968',
 '1969',
 '1970',
 '1970s',
 '1971',
 '1972',
 '1973',
 '1974',
 '1976',
 '1977',
 '1978',
 '1979',
 '1980',
 '1980s',
 '1981',
 '1982',
 '1984',
 '1985',
 '1986',
 '1987',
 '1988',
 '1989',
 '1990s',
 '1990sera',
 '1993',
 '1996',
 '1997',
 '1998',
 '19th',
 '19thcentury',
 '19yearold',
 '20',
 '2001',
 '2003',
 '2004',
 '2005',
 '20062007',
 '2008',
 '2009',
 '2010',
 '2011',
 '2013',
 '2018',
 '2019',
 '2074',
 '20th',
 '21',
 '2154',
 '21st',
 '21stcentury',
 '225',
 '24',
 '24hour',
 '25',
 '2500year',
 '26chapter',
 '26yearold',
 '27yearold',
 '28yearold',
 '29',
 '30',
 '30something',
 '30yearolds',
 '33',
 '3yearold',
 '40',
 '40yearold',
 '4300mile',
 '47',
 '50',
 '500',
 '5000',
 '50yearold',
 '51',
 '54',
 '59yearold',
 '60',
 '64',
 '69',
 '6th',
 '70',
 '74th',
 '7year',
 '80',
 '90',
 '90yearold',
 '911'], axis=1, inplace=True)

In [664]:
Df_vector.head(2)

,title,rated,released,runtime,director,writer,actors,plot,language,country,...,Mark Ruffalo,Nicolas Cage,Chloë Grace Moretz,Johnny Depp,Kristen Stewart,Amy Adams,Robert De Niro,Chris Pine,new,Vectorized
0,Inception,PG-13,2010-07-16,148,Christopher Nolan,Christopher Nolan,Leonardo DiCaprio Joseph Gordon-Levitt Elliot ...,A thief who steals corporate secrets through t...,English Japanese French,United Kingdom United States,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,A thief steal corporate secret use dreamsharin...,"[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,..."
1,Iron Man 2,PG-13,2010-05-07,124,Jon Favreau,Justin Theroux Stan Lee Don Heck,Robert Downey Jr. Mickey Rourke Gwyneth Paltrow,With the world now aware of his identity as Ir...,English French Russian,United States,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,With world aware identity Iron Man Tony Stark ...,"[[1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1,..."


# Creating the classification for prediction

In [665]:
df3.head()

,aa,aang,aaron,abandon,abandoned,abby,abducted,abduction,ability,able,...,zequi,zero,zeus,zibby,zombie,zone,zoo,zookeeper,zoologist,zuckerberg
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [666]:
Df_vector['label']=''
for index, row in Df_vector.iterrows():
    if (row['imdbrating']<5):
        Df_vector.loc[index, 'label']=0
    elif (row['imdbrating']>=5)&(row['imdbrating']<8):
        Df_vector.loc[index, 'label']=1
    elif (row['imdbrating']>=8):
        Df_vector.loc[index, 'label']=2

In [667]:
Df_vector['label'].value_counts()

1    1622
0      78
2      74
Name: label, dtype: int64

In [668]:
Df_vector['label']=pd.to_numeric(Df_vector['label'])
Df_vector['label'].fillna(0,inplace=True)
Df_vector['label']=Df_vector['label'].astype('int')

# Prepare data for model

In [669]:
Df_vector=Df_vector.drop(['title','released','awards','metascore','imdbrating' ,'imdbvotes','dvd',
           'boxoffice','response','gross', 'year','internet_movie_database_rating',
           'rotten_tomato_rating','metacritic_rating','plot','new'],axis=1)                                                 
Df_vector.columns

Index(['rated', 'runtime', 'director', 'writer', 'actors', 'language',
       'country', 'genre', 'star', 'budget',
       ...
       'Mark Ruffalo', 'Nicolas Cage', 'Chloë Grace Moretz', 'Johnny Depp',
       'Kristen Stewart', 'Amy Adams', 'Robert De Niro', 'Chris Pine',
       'Vectorized', 'label'],
      dtype='object', length=137)

In [670]:
Temp = Df_vector[['Vectorized']]

In [671]:
Temp.head(2)

,Vectorized
0,"[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,..."
1,"[[1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1,..."


In [672]:
Df_vector=Df_vector.drop(['country','language','writer','actors','Vectorized'],axis=1)

In [673]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.ensemble import RandomForestClassifier

In [674]:
df2=Df_vector.drop('label',axis=1)
y=Df_vector['label']

In [675]:
df2.columns

Index(['rated', 'runtime', 'director', 'genre', 'star', 'budget',
       'United States', 'United Kingdom', 'France', 'Canada',
       ...
       'Jessica Chastain', 'Chris Evans', 'Mark Ruffalo', 'Nicolas Cage',
       'Chloë Grace Moretz', 'Johnny Depp', 'Kristen Stewart', 'Amy Adams',
       'Robert De Niro', 'Chris Pine'],
      dtype='object', length=131)

In [676]:
X=pd.get_dummies(df2)
X.shape

(1774, 1271)

In [694]:
Words = df3.columns.to_list()

In [696]:
Words_df = pd.DataFrame(Words)

In [698]:
Words_df.to_json('Wordslist.json')

# Merging the plot with the rest of the features

In [677]:
X = pd.merge(X,df3, left_index=True, right_index=True)

# Model 

In [678]:
X

,runtime,budget_x,United States,United Kingdom,France,Canada,China,Germany,Australia,Japan,...,zequi,zero,zeus,zibby,zombie,zone,zoo,zookeeper,zoologist,zuckerberg
0,148,160000000.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1,124,200000000.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
2,138,80000000.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
3,112,60000000.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0,0,0,0,0,0,0,0,0,0
4,102,80000000.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1769,101,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1770,91,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1771,93,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1772,94,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0


In [679]:
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.3, random_state=1)

In [680]:
scaler=StandardScaler().fit(X_train)
X_train_scaled=scaler.transform(X_train)
X_test_scaled=scaler.transform(X_test)

In [681]:
model=RandomForestClassifier (random_state=1,n_estimators=500)
model.fit(X_train_scaled,y_train)

RandomForestClassifier(n_estimators=500, random_state=1)

In [682]:
training_score = model.score(X_train_scaled, y_train)
testing_score = model.score(X_test_scaled, y_test)

In [683]:
print(f"training_score:{training_score},testing_score:{testing_score}")

training_score:1.0,testing_score:0.9136960600375235


In [684]:
from sklearn.svm import SVC

In [685]:
model=SVC()
model.fit(X_train_scaled,y_train)

SVC()

In [686]:
training_score = model.score(X_train_scaled, y_train)
testing_score = model.score(X_test_scaled, y_test)

In [687]:
print(f"training_score:{training_score},testing_score:{testing_score}")

training_score:0.9250604351329573,testing_score:0.9118198874296435


In [688]:
X_test

,runtime,budget_x,United States,United Kingdom,France,Canada,China,Germany,Australia,Japan,...,zequi,zero,zeus,zibby,zombie,zone,zoo,zookeeper,zoologist,zuckerberg
1693,104,15000000.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1684,89,31000000.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
302,103,45000000.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
108,108,25000000.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
919,120,49000000.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1422,107,30000000.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1387,110,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
537,84,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
314,87,60000000.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0


In [689]:
Pred = [X_test.iloc[1]]

In [690]:
Pred

[runtime                 89.0
 budget_x          31000000.0
 United States            1.0
 United Kingdom           0.0
 France                   0.0
                      ...    
 zone                     0.0
 zoo                      0.0
 zookeeper                0.0
 zoologist                0.0
 zuckerberg               0.0
 Name: 1684, Length: 8798, dtype: float64]

In [691]:

scaler=StandardScaler().fit(Pred)
Pred_scaled=scaler.inverse_transform(Pred)



In [692]:
t = model.predict(Pred_scaled)

In [693]:
t

array([1])